In [ ]:
# Importar as bibliotecas necessárias
import pandas as pd
import numpy as np
import os
from datetime import datetime
import calendar
from IPython.display import display

# Função para tocar som ao final (substitui `beep` do R)
def beep():
    print("\a")

# Definindo os anos e meses
anos = range(2020, 2023)
meses = range(1, 13)

# Criando o dataframe expandido com todas as combinações de ano e mês
df_completo = pd.DataFrame([(mes, ano) for ano in anos for mes in meses], columns=['mes', 'ano'])
receita_atualizada = df_completo.sort_values(by=['ano', 'mes']).reset_index(drop=True)

# Caminhos para as pastas onde estão os arquivos CSV
todas_pasta = [
    "E:/Dropbox/transito/microdados/2019", 
    "E:/Dropbox/transito/microdados/2020/corrigido",
    "E:/Dropbox/transito/microdados/2021/corrigido"
]

# Iterar sobre cada pasta
for pasta in todas_pasta:
    # Listar todos os arquivos na pasta que terminam com '.csv'
    arquivos_csv = [os.path.join(pasta, f) for f in os.listdir(pasta) if f.endswith('.csv')]
    
    # Iterar sobre cada arquivo CSV na pasta
    for arquivo in arquivos_csv:
        data = pd.read_csv(arquivo)
        
        # Converter a coluna de data para datetime
        data['data_pagamento'] = pd.to_datetime(data['data_pagamento'], format='%Y-%m-%d')
        
        # Calcular o ano e o mês e filtrar para anos de interesse
        soma = (
            data
            .assign(ano=data['data_pagamento'].dt.year, mes=data['data_pagamento'].dt.month)
            .query("ano in [2020, 2021, 2022]")  # Filtrar anos de interesse
            .groupby(['mes', 'ano'], as_index=False)
            .agg(total_valor_pago=('valor_pago', 'sum'))
        )
        
        # Adicionar ao dataframe principal usando merge
        receita_atualizada = receita_atualizada.merge(soma, on=['mes', 'ano'], how='left')
        
# Emitir som de conclusão
beep()

# Calcular a soma das colunas (excluindo as de ano e mês) e reorganizar o dataframe
receita_atualizada['soma_colunas'] = receita_atualizada.filter(like='total_valor_pago').sum(axis=1)
receita_atualizada = receita_atualizada[['mes', 'ano', 'soma_colunas']]

# Exibir o dataframe final
display(receita_atualizada)
